In [1]:
import os
import gc
import numpy as np
import pandas as pd
import json
import torch
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from src.dataset_new import create_datasets_new, create_folds
from src.train import train_model
from src.evaluate import evaluate_all_metrics
from src.utils import *

from src.models import UNet, UNet2, UNet4, UNet8, UNet2Shallow, UNet4Shallow, UNet8Shallow
from src.model_parts import SingleConv, DoubleConv, TripleConv

c:\Users\wittd\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


c:\Users\wittd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Is tiling and overlap rate okay if the input and mask scale is different
def test_tiling(config):
    if config['tiling']:
        tiling_rate = config['tiling_ratio']
        overlap_rate = config['overlap_rate']
        mask_size = 80 * config['mask_scale']
        input_size = 80 * config['input_scale']
        bio = torch.zeros((config['biosensor_length'], input_size, input_size))
        mask = torch.zeros((mask_size, mask_size))
        bio_tiles, mask_tiles = create_tiles(bio, mask, tiling_rate, overlap_rate)
        print(f"Tile test\nBio tiles: {bio_tiles.shape}, Mask tiles: {mask_tiles.shape}\n")
        assert bio_tiles.shape[0] == mask_tiles.shape[0], "Tile number mismatch"

In [3]:
def run_config(config):
    root_path = os.path.join(config['save_path'], config['run_name'])
    os.makedirs(root_path, exist_ok=True)

    save_config = config.copy()
    save_config['mask_type'] = config['mask_type'].__name__
    save_config['model'] = config['model'].__name__
    save_config['down_conv'] = config['down_conv'].__name__
    save_config['up_conv'] = config['up_conv'].__name__
    json.dump(save_config, open(os.path.join(root_path, 'config.json'), 'w'), indent=4)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device {device}')

    torch.manual_seed(42)
    np.random.seed(42)

    folds, test_dataset = create_folds(config)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    print("Datasets created!")

    for i in range(config['k']):
        print(f'Starting run {i+1}...')
        run_path = os.path.join(root_path, f'run_{i+1}')
        os.makedirs(run_path, exist_ok=True)

        plot_path = os.path.join(run_path, 'plots')
        os.makedirs(plot_path, exist_ok=True)
        model_checkpoint_path = os.path.join(run_path, 'model_checkpoints')
        os.makedirs(model_checkpoint_path, exist_ok=True)

        mean, std, train_dataset, val_dataset = folds[i]
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

        model = config['model'](n_channels=config['biosensor_length'], n_classes=1, 
                                down_conv=config['down_conv'], up_conv=config['up_conv'], 
                                mean=mean, std=std, bilinear=config['bilinear'])
        print(model.__class__.__name__)

        if i == 0:
            # Save model summary only once
            model_summary = summary(model, depth=5)
            with open(os.path.join(root_path, 'model_summary.txt'), 'w') as f:
                f.write(str(model_summary))

        model = model.to(device)

        try:
            log = train_model(
                model,
                device,
                train_loader,
                val_loader,
                config,
                amp=True,
                checkpoint_dir=model_checkpoint_path,
                wandb_dir=root_path,
            )
        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache()
            print('Detected OutOfMemoryError!')

        # Save training log
        json.dump(log, open(os.path.join(run_path, 'log.json'), 'w'), indent=4)
        print()
        best_epoch = find_best_epoch(log)
        print(f'\nBest epoch: {best_epoch}')
        best_checkpoint = torch.load(os.path.join(model_checkpoint_path, f'checkpoint_epoch{best_epoch}.pth'))
        lr = best_checkpoint.pop('learning_rate')
        model.load_state_dict(best_checkpoint)
        model = model.to(device)

        best_model_path = os.path.join(run_path, 'best_model.pth')
        torch.jit.script(model).save(best_model_path)

        test_metrics = evaluate_all_metrics(model, test_loader, device, config['mask_scale'])
        print('Test metrics:', json.dumps(test_metrics, indent=4))
        json.dump(test_metrics, open(os.path.join(root_path, f'test_result_run{i+1}.json'), 'w'), indent=4)

        # plot test results
        if config['tiling']:
            plot_results_tiles(test_loader, model, device, plot_path, 'Test image', config)
        else:
            plot_results_image(test_loader, model, device, plot_path, 'Test image', config)

        del train_loader, val_loader
        del train_dataset, val_dataset
        del model
        torch.cuda.empty_cache()
        gc.collect()

        print(f'Run {i+1} finished!')

    del test_loader, test_dataset
    torch.cuda.empty_cache()
    gc.collect()

    print('All runs finished!')

In [4]:
config = {
    'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNet_len1_doubleconv',
    'project_name': 'UNet',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNet,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 1,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 1,
    
    'batch_size': 26,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': False,
    'tiling_ratio': 10,
    'overlap_rate': 0.25,

    'dilation': 0,
}

# test_tiling(config)
# run_config(config)

In [ ]:
config3 = config.copy()
config3['run_name'] = 'UNet4_len8_singleconv'
config3['project_name'] = 'UNet4'
config3['model'] = UNet4
config3['down_conv'] = config3['up_conv'] = SingleConv
config3['mask_scale'] = 4
config3['biosensor_length'] = 8
run_config(config3)

In [ ]:
config4 = config3.copy()
config4['run_name'] = 'UNet4_len8_doubleconv'
config4['down_conv'] = config4['up_conv'] = DoubleConv
run_config(config4)

In [ ]:
config5 = config3.copy()
config5['run_name'] = 'UNet4_len1_singleconv'
config5['biosensor_length'] = 1
run_config(config5)

In [ ]:
config6 = config.copy()
config6['run_name'] = 'UNet8_len8_singleconv'
config6['project_name'] = 'UNet8'
config6['model'] = UNet8
config6['down_conv'] = config6['up_conv'] = SingleConv
config6['mask_scale'] = 8
config6['biosensor_length'] = 8
run_config(config6)

In [ ]:
config7 = config6.copy()
config7['run_name'] = 'UNet8_len8_doubleconv'
config7['down_conv'] = config7['up_conv'] = DoubleConv
run_config(config7)